<a href="https://colab.research.google.com/github/ushakiet01/FMML_PROJECTS_AND_ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits can provide a more robust and reliable estimate of your model's performance. This technique is commonly known as cross-validation. Cross-validation involves splitting your dataset into multiple subsets (folds), training and validating your model on different combinations of these subsets, and then averaging the performance metrics.

By doing this, you reduce the impact of the randomness introduced by a single train-test split. It helps in obtaining a more stable estimate of your model's generalization performance, especially when your dataset is limited.

Common types of cross-validation include k-fold cross-validation and stratified k-fold cross-validation. In k-fold cross-validation, the dataset is divided into k subsets, and the model is trained and validated k times, each time using a different fold as the validation set and the remaining folds as the training set.

In summary, using cross-validation, along with averaging the performance metrics, is a good practice to get a more reliable assessment of your model's accuracy and to ensure that your results are not overly influenced by the randomness in the initial data split.








2.Does it give more accurate estimate of test accuracy?
Cross-validation provides a more robust estimate of your model's performance on unseen data compared to a single train-test split. While it can give you a more accurate estimate of test accuracy within the context of your dataset, it's essential to note that the performance on the validation set (or cross-validation folds) may not perfectly reflect the performance on a completely independent test set.

Cross-validation helps mitigate the impact of variability in the initial data split, providing a more stable estimate of your model's generalization performance. However, the true test of a model's accuracy is how well it performs on entirely new, unseen data.

If possible, having a separate test set, not used during training or cross-validation, is ideal for obtaining a more unbiased evaluation of your model's performance on completely new samples. This ensures that your model hasn't memorized patterns specific to the validation or training data and can generalize well to unseen instances.

In summary, while cross-validation provides a more accurate estimate of test accuracy within the constraints of your dataset, having a dedicated, independent test set is crucial for a more trustworthy evaluation of your model's real-world performance.








3.What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of iterations, or folds, in cross-validation can have an impact on the estimate of your model's performance. In general, using a higher number of iterations can provide a more reliable estimate, but there are trade-offs to consider.

Advantages of Higher Iterations:

Increased Robustness: With more iterations, your model is evaluated on a larger number of different train-test splits, reducing the impact of randomness in a single split. This leads to a more robust estimate of performance.

Better Utilization of Data: With more folds, you make better use of your available data for both training and validation, potentially capturing a broader range of patterns in the dataset.

Considerations:

Computational Cost: Using a higher number of iterations comes with increased computational cost. Training and evaluating the model multiple times can be time-consuming, especially if you have a large dataset or complex model.

Diminishing Returns: While increasing the number of iterations generally improves robustness, there's a point of diminishing returns. The improvements in estimate stability may become marginal beyond a certain number of iterations.

In practice, common choices for the number of iterations include 5-fold or 10-fold cross-validation. The choice often depends on the size of your dataset, available computational resources, and the desired balance between robust estimation and computational efficiency.

It's a good idea to experiment with different numbers of iterations and observe how your model's performance metrics stabilize. Choose a number that provides a good balance between robustness and practicality for your specific scenario.






4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations in cross-validation can help mitigate the impact of having a very small training or validation dataset to some extent. When your dataset is limited, the model may not have enough examples to learn robust patterns, and a single train-test split may not be representative of the overall data distribution.

By using more iterations in cross-validation, you expose your model to different subsets of the data for training and validation in each iteration. This helps in better leveraging the available data, providing a more stable estimate of performance, and reducing the influence of the specific randomness in a single split.

However, it's important to note the following considerations:

Limited Diversity: Increasing the number of iterations won't magically create more diversity in your small dataset. It helps in utilizing the existing data more effectively, but it can't replace the need for a diverse and representative dataset.

Computational Cost: As mentioned before, higher iterations come with increased computational cost. This might be a concern, especially if you have resource limitations.

Overfitting Risk: If your dataset is extremely small, there's still a risk of overfitting to the specific patterns present in the small dataset, even with cross-validation. It's crucial to be cautious and interpret results accordingly.

In summary, while increasing the number of iterations can be a helpful strategy, it's not a complete solution for dealing with extremely small datasets. It should be combined with other techniques, such as careful feature selection, regularization, and, if possible, acquiring more diverse data to enhance model generalization.




